In [85]:
from surprise import KNNBasic
from surprise import KNNBaseline
from surprise import Dataset,SVD,NormalPredictor
from surprise import Reader
from collections import defaultdict
from surprise.model_selection import GridSearchCV
from operator import itemgetter
import heapq
import os
import csv
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_squared_error


In [86]:
def loadDataSet(fileName):
    reader = Reader(line_format="user item rating timestamp",sep="\t",skip_lines=1)
    ratings_set = Dataset.load_from_file(fileName,reader=reader)
    return ratings_set

In [87]:
trainData = loadDataSet("u1.base")
testData = loadDataSet("u1.test")
testDataSet = testData.build_full_trainset()
trainDataSet = trainData.build_full_trainset()
cols=["user_id","movie_id","ratings","timestamp"]
df=pd.read_csv("u1.base",sep="\t",names=cols,encoding="latin-1")
test_df=pd.read_csv("u1.test",sep="\t",names=cols,encoding="latin-1")
user_user_df=pd.read_csv("u1.test",sep="\t",names=cols,encoding="latin-1")
max_user_id = np.max(test_df.user_id.values)
max_movie_id = np.max(test_df.movie_id.values)
ratings_pivot = df.pivot(index='movie_id', columns='user_id', values="ratings").fillna(0)

In [88]:
cosine_similarity_matrix = KNNBaseline(sim_options={
        'name': 'cosine',
        'user_based': False
        })\
        .fit(trainDataSet)\
        .compute_similarities()

pearson_similarity_matrix = KNNBaseline(sim_options={
        'name': 'pearson_baseline',
        'user_based': False
        })\
        .fit(trainDataSet)\
        .compute_similarities()

cosine_similarity_matrix_user = KNNBaseline(sim_options={
        'name': 'cosine',
        'user_based': True
        })\
        .fit(trainDataSet)\
        .compute_similarities()

pearson_similarity_matrix_user = KNNBaseline(sim_options={
        'name': 'pearson_baseline',
        'user_based': True
        })\
        .fit(trainDataSet)\
        .compute_similarities()

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [71]:
kNN=[2,5,10]

cosine_similarity_matrix = np.nan_to_num(cosine_similarity_matrix);
for user_id in test_df.user_id.values:
    for k in kNN:
        test_subj_iid =testDataSet.to_inner_uid(str(user_id))
        test_subj_ratings = testDataSet.ur[test_subj_iid]
        near_neighbours = heapq.nlargest(k,test_subj_ratings,key=lambda t :t[1])
        itemsDict = defaultdict(float)
        pdKey = str(k)+'_cosine';
        for movieId,rating in near_neighbours:
            sim = np.nan_to_num(cosine_similarity_matrix[movieId]);
            for innerId,simScore in enumerate(sim):
                itemsDict[innerId] += simScore*(rating/5.0);
            test_df.loc[((test_df["movie_id"]==movieId) & (test_df["user_id"]==user_id)),pdKey] =itemsDict[movieId]
        test_df[pdKey] = test_df[pdKey].fillna(0)
            

pearson_similarity_matrix = np.nan_to_num(pearson_similarity_matrix);
for user_id in test_df.user_id.values:
    for k in kNN:
        test_subj_iid =testDataSet.to_inner_uid(str(user_id))
        test_subj_ratings = testDataSet.ur[test_subj_iid]
        near_neighbours = heapq.nlargest(k,test_subj_ratings,key=lambda t :t[1])
        itemsDict = defaultdict(float)
        pdKey = str(k)+'_pearson';
        for movieId,rating in near_neighbours:
            sim = np.nan_to_num(pearson_similarity_matrix[movieId]);
            for innerId,simScore in enumerate(sim):
                itemsDict[innerId] += simScore*(rating/5.0);
            test_df.loc[((test_df["movie_id"]==movieId) & (test_df["user_id"]==user_id)),pdKey] =itemsDict[movieId]
        test_df[pdKey] = test_df[pdKey].fillna(0)

     
cosine_similarity_matrix_user = np.nan_to_num(cosine_similarity_matrix);
for movie_id in test_df.movie_id.values:
    for k in kNN:
        test_subj_iid =testDataSet.to_inner_iid(str(movie_id))
        test_subj_ratings = testDataSet.ir[test_subj_iid]
        near_neighbours = heapq.nlargest(k,test_subj_ratings,key=lambda t :t[1])
        itemsDict = defaultdict(float)
        pdKey = str(k)+'_cosine';
        for userId,rating in near_neighbours:
            sim = np.nan_to_num(cosine_similarity_matrix_user[userId]);
            for innerId,simScore in enumerate(sim):
                itemsDict[innerId] += simScore*(rating/5.0);
            user_user_df.loc[((user_user_df["movie_id"]==movie_id) & (user_user_df["user_id"]==userId)),pdKey] =itemsDict[userId]
        user_user_df[pdKey] = user_user_df[pdKey].fillna(0)
            
            

pearson_similarity_matrix_user = np.nan_to_num(pearson_similarity_matrix_user);
for movie_id in test_df.movie_id.values:
    for k in kNN:
        test_subj_iid =testDataSet.to_inner_iid(str(movie_id))
        test_subj_ratings = testDataSet.ir[test_subj_iid]
        near_neighbours = heapq.nlargest(k,test_subj_ratings,key=lambda t :t[1])
        itemsDict = defaultdict(float)
        pdKey = str(k)+'_pearson';
        for userId,rating in near_neighbours:
            sim = np.nan_to_num(pearson_similarity_matrix_user[userId]);
            for innerId,simScore in enumerate(sim):
                itemsDict[innerId] += simScore*(rating/5.0);
            user_user_df.loc[((user_user_df["movie_id"]==movie_id) & (user_user_df["user_id"]==userId)),pdKey] =itemsDict[userId]
        user_user_df[pdKey] = user_user_df[pdKey].fillna(0)

In [78]:
print("********************** QUESTION 5 ********************************************************")
print("************ Movie Movie  ***********************")
print("Cosine Movie Movie Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['2_cosine'],test_df['ratings'])))
print("Cosine Movie Movie Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['5_cosine'],test_df['ratings'])))
print("Cosine Movie Movie Top10_RMSE_Score : ",math.sqrt(mean_squared_error(test_df['10_cosine'],test_df['ratings'])))
print("Pearson Movie Movie Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['2_pearson'],test_df['ratings'])))
print("Pearson Movie Movie Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['5_pearson'],test_df['ratings'])))
print("Pearson Movie Movie Top10_RMSE_Score : ",math.sqrt(mean_squared_error(test_df['10_pearson'],test_df['ratings'])))

print("************ user user  ***********************")
print("Cosine User User Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_df['2_cosine'],user_user_df['ratings'])))
print("Cosine User User Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_df['5_cosine'],user_user_df['ratings'])))
print("Cosine User User Top10_RMSE_Score : ",math.sqrt(mean_squared_error(user_user_df['10_cosine'],user_user_df['ratings'])))
print("Pearson User User Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_df['2_pearson'],user_user_df['ratings'])))
print("Pearson User User Top5_RMSE_Score  : ",math.sqrt(mean_squared_error(user_user_df['5_pearson'],user_user_df['ratings'])))
print("Pearson User User Top10_RMSE_Score : ",math.sqrt(mean_squared_error(user_user_df['10_pearson'],user_user_df['ratings'])))
print("\n")


********************** QUESTION 5 ********************************************************
************ Movie Movie  ***********************
Cosine Movie Movie Top2_RMSE_Score  :  3.716501918508152
Cosine Movie Movie Top5_RMSE_Score  :  3.7093519570772995
Cosine Movie Movie Top10_RMSE_Score :  3.707645599799954
Pearson Movie Movie Top2_RMSE_Score  :  3.7172478607171935
Pearson Movie Movie Top5_RMSE_Score  :  3.7137808683872695
Pearson Movie Movie Top10_RMSE_Score :  3.708120109297158
************ user user  ***********************
Cosine User User Top2_RMSE_Score  :  3.71356869516944
Cosine User User Top5_RMSE_Score  :  3.702847249394114
Cosine User User Top10_RMSE_Score :  3.696532293087497
Pearson User User Top2_RMSE_Score  :  3.7150783705798296
Pearson User User Top5_RMSE_Score  :  3.7102318277012767
Pearson User User Top10_RMSE_Score :  3.701977170572221




In [5]:
param_grid = {'n_factors':[2,5,10],'n_epochs':[10,20,30],  'lr_all':[0.0005,0.001,0.005,0.01],'reg_all':[0.001,0.003,0.05,0.5,0.75]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(trainData)

In [6]:
gs.best_params['rmse']

{'n_factors': 5, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.05}

In [107]:
kFactors=[2,5,10]
for factor in kFactors:
    svd = SVD(n_factors=factor,n_epochs=30,lr_all=0.01,reg_all=0.05)
    svd.fit(trainDataSet)
    key='{}Latent_Factor'.format(factor)
    print("************ {} ************* ".format(key))
    for user_id  in  test_df.user_id.unique():
        for movie_id in test_df.movie_id.unique():
            test_df.loc[((test_df["movie_id"]==movie_id) & (test_df["user_id"]==user_id)),key] =str(svd.predict(uid=user_id,iid=movie_id).est)
        

************ 2Latent_Factor ************* 
************ 5Latent_Factor ************* 
************ 10Latent_Factor ************* 


In [110]:
print("********************** QUESTION 5 ********************************************************")
print("************ Movie Movie  ***********************")
print("Cosine Movie Movie Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['2Latent_Factor'],test_df['ratings'])))
print("Cosine Movie Movie Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['5Latent_Factor'],test_df['ratings'])))
print("Cosine Movie Movie Top2_RMSE_Score  : ",math.sqrt(mean_squared_error(test_df['10Latent_Factor'],test_df['ratings'])))


********************** QUESTION 5 ********************************************************
************ Movie Movie  ***********************
Cosine Movie Movie Top2_RMSE_Score  :  1.1536760683206604
Cosine Movie Movie Top2_RMSE_Score  :  1.1536760683206604
Cosine Movie Movie Top2_RMSE_Score  :  1.1536760683206604
